# Data Science z Python 3.10. Konwersja modeli (fast.ai & TensorFlow)
## 🇬🇧 Data Science with Python 3.10. Model conversion (fast.ai & TensorFlow)
#### 👨‍🏫 PhD Wojciech Oronowicz-Jaśkowiak
#### 🤖 https://github.com/aipogodzinach

## Zadanie 1.
### Task 1.

In [1]:
# pip install numpy keras tensorflow tf2onnx

import os
import numpy as np
import shutil
import keras

from sklearn.metrics import confusion_matrix, classification_report
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.applications import ResNet50

In [ ]:
base_dir = 'dataset_x5'
data_dir = 'data_dir'

raw_no_of_files = {}
classes = ['class1', 'class2', 'class3']
for dir in classes:
    raw_no_of_files[dir] = len(os.listdir(os.path.join(base_dir, dir)))
raw_no_of_files.items()

dict_items([('class1', 111), ('class2', 114), ('class3', 136)])

In [ ]:
if not os.path.exists(data_dir):
    os.mkdir(data_dir)

train_dir = os.path.join(data_dir, 'train')
valid_dir = os.path.join(data_dir, 'valid')
test_dir = os.path.join(data_dir, 'test')

train_class1_dir = os.path.join(train_dir, '1')
train_class2_dir = os.path.join(train_dir, '2')
train_class3_dir = os.path.join(train_dir, '3')

valid_class1_dir = os.path.join(valid_dir, '1')
valid_class2_dir = os.path.join(valid_dir, '2')
valid_class3_dir = os.path.join(valid_dir, '3')

test_class1_dir = os.path.join(test_dir, '1')
test_class2_dir = os.path.join(test_dir, '2')
test_class3_dir = os.path.join(test_dir, '3')

for directory in (train_dir, valid_dir, test_dir):
    if not os.path.exists(directory):
        os.mkdir(directory)

dirs = [train_class1_dir, train_class2_dir, train_class3_dir,
        valid_class1_dir, valid_class2_dir, valid_class3_dir,
        test_class1_dir, test_class2_dir, test_class3_dir]

for dir in dirs:
    if not os.path.exists(dir):
        os.mkdir(dir)        

In [ ]:
class1_fnames = os.listdir(os.path.join(base_dir, 'class1'))
class2_fnames = os.listdir(os.path.join(base_dir, 'class2'))
class3_fnames = os.listdir(os.path.join(base_dir, 'class3'))

class1_fnames = [fname for fname in class1_fnames if fname.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]
class2_fnames = [fname for fname in class2_fnames if fname.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]
class3_fnames = [fname for fname in class3_fnames if fname.split('.')[1].lower() in ['jpg', 'png', 'jpeg']]

In [ ]:
size = min(len(class1_fnames), len(class2_fnames), len(class3_fnames))

train_size = int(np.floor(0.7 * size))
valid_size = int(np.floor(0.2 * size))
test_size = size - train_size - valid_size

train_idx = train_size
valid_idx = train_size + valid_size
test_idx = train_size + valid_size + test_size

In [ ]:
for i, fname in enumerate(class1_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'class1', fname)
        dst = os.path.join(train_class1_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'class1', fname)
        dst = os.path.join(valid_class1_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'class1', fname)
        dst = os.path.join(test_class1_dir, fname)
        shutil.copyfile(src, dst)

for i, fname in enumerate(class2_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'class2', fname)
        dst = os.path.join(train_class2_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'class2', fname)
        dst = os.path.join(valid_class2_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'class2', fname)
        dst = os.path.join(test_class2_dir, fname)
        shutil.copyfile(src, dst) 

for i, fname in enumerate(class3_fnames):
    if i <= train_idx:
        src = os.path.join(base_dir, 'class3', fname)
        dst = os.path.join(train_class3_dir, fname)
        shutil.copyfile(src, dst)
    elif train_idx < i <= valid_idx:
        src = os.path.join(base_dir, 'class3', fname)
        dst = os.path.join(valid_class3_dir, fname)
        shutil.copyfile(src, dst)
    elif valid_idx < i < test_idx:
        src = os.path.join(base_dir, 'class3', fname)
        dst = os.path.join(test_class3_dir, fname)
        shutil.copyfile(src, dst)        

In [ ]:
train_datagen = ImageDataGenerator()

valid_datagen = ImageDataGenerator(rescale=1./255.)

train_generator = train_datagen.flow_from_directory(directory=train_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='categorical')

valid_generator = valid_datagen.flow_from_directory(directory=valid_dir,
                                                   target_size=(150, 150),
                                                   batch_size=32,
                                                   class_mode='categorical')

Found 234 images belonging to 3 classes.
Found 66 images belonging to 3 classes.


In [ ]:
batch_size = 20
steps_per_epoch = train_size // batch_size
validation_steps = valid_size // batch_size

In [ ]:
conv_base = ResNet50(weights='imagenet', include_top=False, input_shape=(150, 150, 3))
conv_base.trainable = True

def print_layers(model):
    for layer in model.layers:
        print(f'layer_name: {layer.name:13} trainable: {layer.trainable}')

print_layers(conv_base)

94765736/94765736 [==============================] - 8s 0us/step
layer_name: input_1       trainable: True
layer_name: conv1_pad     trainable: True
layer_name: conv1_conv    trainable: True
layer_name: conv1_bn      trainable: True
layer_name: conv1_relu    trainable: True
layer_name: pool1_pad     trainable: True
layer_name: pool1_pool    trainable: True
layer_name: conv2_block1_1_conv trainable: True
layer_name: conv2_block1_1_bn trainable: True
layer_name: conv2_block1_1_relu trainable: True
layer_name: conv2_block1_2_conv trainable: True
layer_name: conv2_block1_2_bn trainable: True
layer_name: conv2_block1_2_relu trainable: True
layer_name: conv2_block1_0_conv trainable: True
layer_name: conv2_block1_3_conv trainable: True
layer_name: conv2_block1_0_bn trainable: True
layer_name: conv2_block1_3_bn trainable: True
layer_name: conv2_block1_add trainable: True
layer_name: conv2_block1_out trainable: True
layer_name: conv2_block2_1_conv trainable: True
layer_name: conv2_block2_1_bn t

In [ ]:
set_trainable = False
for layer in conv_base.layers:
    if layer.name == 'block5_conv1':
        set_trainable = True
    if set_trainable:
        layer.trainable = True
    else:
        layer.trainable = False

print_layers(conv_base)

layer_name: input_1       trainable: False
layer_name: conv1_pad     trainable: False
layer_name: conv1_conv    trainable: False
layer_name: conv1_bn      trainable: False
layer_name: conv1_relu    trainable: False
layer_name: pool1_pad     trainable: False
layer_name: pool1_pool    trainable: False
layer_name: conv2_block1_1_conv trainable: False
layer_name: conv2_block1_1_bn trainable: False
layer_name: conv2_block1_1_relu trainable: False
layer_name: conv2_block1_2_conv trainable: False
layer_name: conv2_block1_2_bn trainable: False
layer_name: conv2_block1_2_relu trainable: False
layer_name: conv2_block1_0_conv trainable: False
layer_name: conv2_block1_3_conv trainable: False
layer_name: conv2_block1_0_bn trainable: False
layer_name: conv2_block1_3_bn trainable: False
layer_name: conv2_block1_add trainable: False
layer_name: conv2_block1_out trainable: False
layer_name: conv2_block2_1_conv trainable: False
layer_name: conv2_block2_1_bn trainable: False
layer_name: conv2_block2_1_re

In [ ]:
model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(units=256, activation='relu'))
model.add(layers.Dense(units=3, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(learning_rate=1e-5),
             loss='categorical_crossentropy',
             metrics=['accuracy'])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 5, 5, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 51200)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               13107456  
                                                                 
 dense_3 (Dense)             (None, 3)                 771       
                                                                 
Total params: 36,695,939
Trainable params: 13,108,227
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
model.fit(train_generator, epochs=1, steps_per_epoch=2)

2/2 [==============================] - 4s 914ms/step - loss: 2.2320 - accuracy: 0.4286


In [ ]:
model.save('C:\\Users\\s21072\\my_model_jupyter')

INFO:tensorflow:Assets written to: C:\Users\s21072\my_model_jupyter\assets


INFO:tensorflow:Assets written to: C:\Users\s21072\my_model_jupyter\assets


In [ ]:
new_model = keras.models.load_model('C:\\Users\\s21072\\my_model_jupyter')
new_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 5, 5, 2048)        23587712  
                                                                 
 flatten_1 (Flatten)         (None, 51200)             0         
                                                                 
 dense_2 (Dense)             (None, 256)               13107456  
                                                                 
 dense_3 (Dense)             (None, 3)                 771       
                                                                 
Total params: 36,695,939
Trainable params: 13,108,227
Non-trainable params: 23,587,712
_________________________________________________________________


In [ ]:
#python -m tf2onnx.convert --saved-model C:/Users/s21072/my_model_jupyter --output C:/Users/s21072/my_model_jupyter.onnx

## Zadanie 2.
### Task 2.

In [3]:
!pip install fastai==2.5.3 fastbook==0.0.18 torch==1.10.0 torchvision==0.11.1 seeme

from fastbook import *
from fastai.vision.widgets import *

import seeme
from seeme import Client

import pathlib
import fastai
from pathlib import Path

import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/My Drive/dataset_x5')
path = Path('/content/drive/My Drive/dataset_x5')

classes = 'class1', 'class2', 'class3'

data = DataBlock(
    blocks=(ImageBlock, CategoryBlock),
    get_items=get_image_files,
    splitter=RandomSplitter(valid_pct=0.2,seed=42),
    get_y=parent_label,
    item_tfms=Resize(128)
)

data = data.new(
    item_tfms=RandomResizedCrop(224, min_scale=0.5),
    batch_tfms=aug_transforms(mult=0.0, do_flip=False, flip_vert=False, max_rotate=0.0, min_zoom=0.0, max_zoom=0.0, max_lighting=0.0, max_warp=0.0, p_affine=0.0, p_lighting=0.0, xtra_tfms=None, size=None, mode='bilinear', pad_mode='border', align_corners=True, batch=False, min_scale=1.0))

dls = data.dataloaders(path, bs = 32)

learn = cnn_learner(dls, resnet18, metrics=error_rate)

learn.fit_one_cycle(1)
learn.export(fname="model.pkl")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 189 kB 22.4 MB/s 
     |████████████████████████████████| 720 kB 68.2 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x2e84000 @  0x7f7999ac9615 0x5d6f4c 0x51edd1 0x51ef5b 0x4f750a 0x4997a2 0x4fd8b5 0x4997c7 0x4fd8b5 0x49abe4 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x4f5fe9 0x55e146 0x5d8868 0x5da092 0x587116 0x5d8d8c 0x55dc1e 0x55cd91 0x5d8941 0x49abe4 0x55cd91 0x5d8941 0x4990ca 0x5d8868 0x4997a2 0x4fd8b5 0x49abe4
     |████████████████████████████████| 881.9 MB 3.7 kB/s 
     |████████████████████████████████| 23.3 MB 1.3 MB/s 
     |████████████████████████████████| 55 kB 5.4 MB/s 
     |████████████████████████████████| 64 kB 3.6 MB/s 
     |████████████████████████████████| 1.3 MB 66.8 MB/s 
     |████████████████████████████████| 58 kB 6.4 MB/s 
     |███████████████████████████

Mounted at /content/drive


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

epoch,train_loss,valid_loss,error_rate,time
0,0.801419,0.120980,0.044776,01:37


In [ ]:
"""
learn.model.eval();
learn.export()

my_password = ""
my_username = ""
my_email = ""
my_firstname = ""
my_name = ""

cl.register(username=my_username, email=my_email, password=my_password, firstname=my_firstname, name=my_name)

cl.login(my_username, my_password);

application_id = cl.get_application_id(
    base_framework="pytorch",
    framework="fastai",
    base_framework_version=str(torch.__version__),
    framework_version=str(fastai.__version__),
    application="image_classification"
)

model_name = ""
description = ""

my_model = cl.create_model(  {
    "name": model_name,
    "description": description,
    "application_id": application_id,
    "auto_convert": True
})

cl.upload_model(my_model["id"], str(learn.path))

"""

# app.seeme.ai

In [4]:
import torch as torch

model_file = open("model.pkl", 'rb')
model = torch.load(model_file, map_location=torch.device('cpu'))

model_eval = model.eval()
dummy_image = torch.randn(1, 3, 64, 64)
torch.onnx.export(model_eval, dummy_image, "model.onnx")